# Programming Assignment 2: Bigram Language Models

In Programming Assignment 1, we are going to implement a bigram language model and use interpolation for smoothing.

The bigram probabilities will be estimated by using MLE as follows:

$$
P_{ML}(w_i|w_{i-1})=\frac{c(w_{i-1}w_{i})}{c(w_{i-1})}
$$

Recall that, the smoothed bigram probabilities using interpolation technique is calculated as follows.

Bigrams:

$$
P(w_i|w_{i-1})=\lambda_2 \times P_{ML}(w_i|w_{i-1})+(1-\lambda_2)\times P(w_i)
$$

where $P(w_i)$ is the smoothed unigram probability and calculated as follows.

$$
P(w_i)=\lambda_1 \times P_{ML}(w_i) + (1-\lambda_1) \times \frac{1}{N}
$$

where $N$ is a large number, e.g., $N=1,000,000$.

There are two parts in this programming assignment.

- Training: you will estimate unigram and bigram probabilities from a raw text data file and save parameters of the language model into a file.
- Test: you use the trained language model to calculate calculates entropy, perplexity on the test data. You will need to use interpolation to calculate smoothed probabilities in testing.

Please refer the pseudo-code in slide 17, 18 of the lecture [NLP Programming Tutorial 2 -Bigram Language Models](http://www.phontron.com/slides/nlp-programming-en-02-bigramlm.pdf) to complete the assignment.

## Submission and due date

**The due for the programming assignment 2 will be at 23:59 on February 19, 2023 (Hard deadline)**

### How to submit

- Attach notebook file (.ipynb) and submit your work to Google Class Room 
- Name your file as YourName_StudentID_Assignment1.ibynb. E.g., Nguyen_Van_A_ST099834_Assignment2.ipynb
- Copying others' assignments is strictly prohibited.
- Write your name and student ID into this notebook


## Data

We will use the file [wiki-en-train.word](https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word) as the training data, and [wiki-en-test.
word](https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word) as the test data. To test our implementation quickly, we will use small data file [02-train-input.txt](https://github.com/neubig/nlptutorial/blob/master/test/02-train-input.txt). All data files are from the [nlptutorial](https://github.com/neubig/nlptutorial) by Graham Neubig.

As the first step, we will download all necessary data files using `wget` command line.

In [ ]:
!rm -f wiki-en-train.word
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word
    
!rm -f wiki-en-test.word
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word

!rm -f 02-train-input.txt
!wget https://raw.githubusercontent.com/neubig/nlptutorial/master/test/02-train-input.txt

--2023-02-20 11:55:02--  https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-train.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203886 (199K) [text/plain]
Saving to: ‘wiki-en-train.word’

wiki-en-train.word  100%[===================>] 199.11K  --.-KB/s    in 0.02s   

2023-02-20 11:55:03 (8.01 MB/s) - ‘wiki-en-train.word’ saved [203886/203886]

--2023-02-20 11:55:03--  https://raw.githubusercontent.com/neubig/nlptutorial/master/data/wiki-en-test.word
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2698

## Part 1: Estimating probabilities (60 points)

What you need to do in this part is complete the function `train_bigram` to estimate unigram, bigram probabilities (using MLE method) from a text file and save probabilities to a model file.

The format of the model file is as follows. Each line includes an n-gram (unigram or bigram) and its probability estimated by MLE method.

```
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
...
```

### Part 1.1: Function train_bigram()

In [ ]:
from collections import defaultdict


def train_bigram(train_file, model_file):
    """Train bigram language model and save to model file
    """
    counts = defaultdict(int)  # count the n-gram
    context_counts = defaultdict(int)   # count the context
    with open(train_file) as f:
        for line in f:
            line = line.lower().strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')

            for i in range(1, len(words)):  # Note: starting at 1, after <s>
                # TODO: Write code to count bigrams and their contexts
                # YOUR CODE HERE
                if not context_counts[words[i]]:
                  context_counts[words[i]] = 1
                else: 
                  context_counts[words[i]] += 1
                bi = words[i-1] + ' ' + words[i]
                if not counts[bi]:
                  counts[bi] = 1
                else: counts[bi] += 1
            if not context_counts["<s>"]:
              context_counts["<s>"] = 1
            else:
              context_counts["<s>"] += 1

    # Save probabilities to the model file            
    with open(model_file, 'w') as fo:
        for ngram, count in counts.items():
            # TODO: Write code to calculate probabilities of n-grams 
            # (unigrams and bigrams)
            # Hint: probabilities of n-grams will be calculated by their counts
            # divided by their context's counts.
            # probability = counts[ngram]/context_counts[context]
            # After calculating probabilities, we will save ngram and probability
            # to the file in the format:
            # ngram<tab>probability
            # YOUR CODE HERE
            context = ngram.split(" ")[0]
            fo.writelines(ngram+"    "+str(count/context_counts[context])+"\n")
        for uni, count in context_counts.items():
            summ = sum(context_counts.values())
            fo.writelines(uni+"    "+str(count/summ)+"\n")

Let's try to train bigram model on the small data.

In [ ]:
train_bigram('02-train-input.txt', '02-train-answer.txt')

Let's see the content of the model. After completing the function `train_bigram`, you should see. The order of lines may be different.

```
</s>	0.250000
<s> a	1.000000
a	0.250000
a b	1.000000
b	0.250000
b c	0.500000
b d	0.500000
c	0.125000
c </s>	1.000000
d	0.125000
d </s>	1.000000
```

In [ ]:
!cat 02-train-answer.txt

<s> a    1.0
a b    1.0
b c    0.5
c </s>    1.0
b d    0.5
d </s>    1.0
a    0.2
b    0.2
c    0.1
</s>    0.2
<s>    0.2
d    0.1


### Part 1.2: load the model file

We are going to implement the function `load_bigram_model` to load the model file.

In [ ]:
def load_bigram_model(model_file):
    """Load the model file

    Args:
        model_file (str): Path to the model file
    
    Returns:
        probs (dict): Dictionary object that map from ngrams to their probabilities
    """
    probs = {}
    with open(model_file, 'r') as f:
        for line in f:
            # TODO: From each line split ngram, probability
            # and then update probs
            a, b = line.split("    ")
            b = float(b.replace('\n', ''))
            probs[a] = b
            # YOUR CODE HERE
            
            pass
    return probs

Let's test the function

In [ ]:
probs = load_bigram_model('02-train-answer.txt')
probs

{'<s> a': 1.0,
 'a b': 1.0,
 'b c': 0.5,
 'c </s>': 1.0,
 'b d': 0.5,
 'd </s>': 1.0,
 'a': 0.2,
 'b': 0.2,
 'c': 0.1,
 '</s>': 0.2,
 '<s>': 0.2,
 'd': 0.1}

## Part 2: Evaluating Bigram Language Model (40 points)

In this part, we will evaluate the bigram language model on the test set. We will use linear interpolation as the smoothing technique.

What we need to do is to complete the function  `test_bigram` as follows. The function will return perplexity on the test data.

Recall that, the smoothed bigram probabilities using interpolation technique is calculated as follows.

Bigrams:

$$
P(w_i|w_{i-1})=\lambda_2 \times P_{ML}(w_i|w_{i-1})+(1-\lambda_2)\times P(w_i)
$$

where $P(w_i)$ is the smoothed unigram probability and calculated as follows.

$$
P(w_i)=\lambda_1 \times P_{ML}(w_i) + (1-\lambda_1) \times \frac{1}{N}
$$

where $N$ is a large number, e.g., $N=1,000,000$.

In [ ]:
import math


def test_bigram(test_file, model_file, lambda2=0.95, lambda1=0.95, N=1000000):
    W = 0 # Total word count
    H = 0
    probs = load_bigram_model(model_file)
    with open(test_file, 'r') as f:
        for line in f:
            line = line.lower().strip()
            if line == '':
                continue
            words = line.split()
            words.append('</s>')
            words.insert(0, '<s>')
            for i in range(1, len(words)):  # Note: starting at 1, after <s>
                # TODO: Write code to calculate smoothed unigram probabilties
                # and smoothed bigram probabilities
                # You should use calculate p1 as smoothed unigram probability
                # and p2 as smoothed bigram probability
                p1 = None
                p2 = None

                # YOUR CODE HERE
                if words[i-1] in probs.keys():
                    prob_uni = float(probs[words[i-1]])
                else: prob_uni = 0
                
                word = words[i-1]+" "+words[i]
                if word in probs.keys():
                    prob_bi = float(probs[word])
                else: prob_bi = 0

                p1 = lambda1*prob_uni+(1-lambda1)*(1/N)

                p2 = lambda2*prob_bi+(1-lambda2)*p1


                # END OF YOUR CODE

                W += 1  # Count the words
                H += -math.log2(p2) # We use logarithm to avoid underflow
    H = H/W
    P = 2**H
    
    print("Entropy: {}".format(H))
    print("Perplexity: {}".format(P))

    return P

Now let's calculate on the Wikipedia data.

In [ ]:
train_bigram('wiki-en-train.word', 'bigram_model.txt')
test_bigram('wiki-en-test.word', 'bigram_model.txt')

Entropy: 10.912193676064902
Perplexity: 1927.0705523728825


1927.0705523728825

You should get entropy value about 11.28.